# MIS 307 Project Phase 4 Final Report
## Group Code: PG1
##### Group Members: Luke Kasner, Cooper Kuhlemeier, Zach Bathje, Cameron Deroos

## 1. Project Description
A local coffee shop is rapidly growing and is in need of a new Point-of-Sale (POS) system. This system needs to provide employees with a user interface that will allow them to track sales, host a loyalty program, and generate weekly and monthly sales reports. The employee’s need to be able to create a sale, attach a loyalty program number, modify the sale, and complete the sale. In addition, they must be able to add users to the loyalty program and check customers’ current  loyalty status. They have requested this information to be stored in a database. This will allow for simple data submission and recovery. The coffee shop’s manager has requested the ability to generate a weekly and monthly report containing information such as the customers who visited most frequently, most popular items sold, and average purchase size and amount in USD. This information will be used to analyze the performance of the business based on customer trends and behaviors. 
## 2. Implementation
### 2.1 Initialization and Importing

In [1]:
#Cell 1
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import time
from datetime import datetime

# Creation of dataframes from csv's 
product_df = pd.read_csv('product.csv')
customer_df = pd.read_csv('customer.csv')
sales_receipts_df = pd.read_csv('sales_reciepts.csv')

# Initializing menu item dictionaries 
coffee = {
    "Small Black Coffee" : 2.00,
    "Medium Black Coffee": 2.50,
    "Large Black Coffee": 3.00,
    "Small Latte" : 3.50,
    "Medium Latte": 4.50,
    "Large Latte": 5.00,
    "Espresso Shot" : 2.00
}

tea = {
    "Small Black Tea": 2.50,
    "Medium Black Tea": 3.50,
    "Large Black Tea": 4.00,
    "Small Green Tea": 2.75,
    "Medium Green Tea": 3.75,
    "Large Green Tea": 4.75,
}
food = {
    "Scone": 5.00,
    "Pumpkin Bread": 4.00,
    "Chocolate Chip Cookie": 2.00
}

In [2]:
#Cell #2

# Writing the product dataframe into a sql file for later query use
with open('product.sql', mode='w') as prod:
    prod.write('DROP TABLE IF EXISTS Product;\n')
    prod.write('CREATE TABLE Product (product_id REAL,product_group TEXT,product_type TEXT,product TEXT,unit_of_measure TEXT, current_retail_price Decimal(4,1));\n')
    prod.write('INSERT INTO Product (product_id,product_group,product_type,product,unit_of_measure,current_retail_price) VALUES\n')
    for i in range(product_df.shape[0]):
        prod.write('(\'' + str(product_df.iloc[i]['product_id'])+'\',')
        prod.write('\''+product_df.iloc[i]['product_group']+'\',')
        prod.write('\''+product_df.iloc[i]['product_type']+'\',')
        prod.write('\''+product_df.iloc[i]['product']+'\',')
        prod.write('\''+product_df.iloc[i]['unit_of_measure']+'\',')
        prod.write(str(product_df.iloc[i]['current_retail_price'])+')')
        if i < product_df.shape[0]-1:
            prod.write(',\n')
        else:
            prod.write(';\n')

In [3]:
#Cell #3
# Establishing product database connection

!sqlite3 product.db < ./product.sql
connProduct = sqlite3.connect('product.db')

In [4]:
#Cell #4
# Writing the customer dataframe into a sql file for later query use

with open('customer.sql', mode='w') as cust:
    cust.write('DROP TABLE IF EXISTS Customer;\n')
    cust.write('CREATE TABLE Customer (customer_id REAL,customer_firstname TEXT,customer_email TEXT,customer_since TEXT,loyalty_card_number TEXT,birthdate TEXT,gender TEXT, points REAL);\n')
    cust.write('INSERT INTO Customer (customer_id,customer_firstname,customer_email,customer_since,loyalty_card_number,birthdate,gender, points) VALUES\n')
    for i in range(customer_df.shape[0]):
        cust.write('(\'' + str(customer_df.iloc[i]['customer_id'])+'\',')
        cust.write('\''+customer_df.iloc[i]['customer_firstname']+'\',')
        cust.write('\''+customer_df.iloc[i]['customer_email']+'\',')
        cust.write('\''+customer_df.iloc[i]['customer_since']+'\',')
        cust.write('\''+customer_df.iloc[i]['loyalty_card_number']+'\',')
        cust.write('\''+customer_df.iloc[i]['birthdate']+'\',')
        cust.write('\''+customer_df.iloc[i]['gender']+'\',')
        cust.write(str(customer_df.iloc[i]['points'])+')')
        if i < customer_df.shape[0]-1:
            cust.write(',\n')
        else:
            cust.write(';\n')

In [5]:
#Cell #5
# Establishing customer database connection

!sqlite3 customer.db < ./customer.sql
connCustomer = sqlite3.connect('customer.db')

In [6]:
#Cell #6
# Writing the sales receipts dataframe into a sql file for later query use

with open('sales_receipts.sql', mode='w') as sale:
    sale.write('DROP TABLE IF EXISTS SalesReceipts;\n')
    sale.write('CREATE TABLE SalesReceipts (transaction_id REAL,transaction_date TEXT,customer_id REAL,product_id REAL,quantity REAL,unit_price Decimal(4,1));\n')
    sale.write('INSERT INTO SalesReceipts (transaction_id,transaction_date,customer_id,product_id,quantity,unit_price) VALUES\n')
    for i in range(sales_receipts_df.shape[0]):
        sale.write('(\'' + str(sales_receipts_df.iloc[i]['transaction_id'])+'\',')
        sale.write('\''+sales_receipts_df.iloc[i]['transaction_date']+'\',')
        sale.write(str(sales_receipts_df.iloc[i]['customer_id'])+',')
        sale.write(str(sales_receipts_df.iloc[i]['product_id'])+',')
        sale.write(str(sales_receipts_df.iloc[i]['quantity'])+',')
        sale.write(str(sales_receipts_df.iloc[i]['unit_price'])+')')
        if i < sales_receipts_df.shape[0]-1:
            sale.write(',\n')
        else:
            sale.write(';\n')

In [7]:
#Cell #7
# Establishing sales receipts database connection

!sqlite3 sales_receipts.db < ./sales_receipts.sql
connSales = sqlite3.connect('sales_receipts.db')

### 2.2 Main selector tree

In [8]:
#Cell #8

# selector is the main function that runs the user interface 
def selector():
    #Sleeps for 1 second to give graphs and insight time to load later on
    time.sleep(1)
    while True:
        print("Please make a selection:")
        action = int(input("1 - Create New Sale \n2 - Loyalty Program \n3 - Management Options\n4 - Shutdown\nInput: "))
        if action == 1:
            print("You have selected 1 - Create New Sale")
            startSale()
            break
        elif action == 2:
            print("You have selected 2 - Loyalty Program")
            loyaltyOptions(connCustomer)
            break
        elif action == 3:
            print("You have selected 3 - Management Options")
            managementOptions()
            break
        elif action == 4:
            print("Shutting down...")
            break
        else:
            print("Please select option 1, 2,3 or 4.")

### 2.3 Main startSale function

In [9]:
# #Cell #9
def startSale():
    # Sale is an isntance of the sales() class. Allows for modification of sale until sale is finalized and sale.Close() is called.
    sale = sales(customer_df, product_df, sales_receipts_df, connSales)
    while True:
        # group_select is the main selection tree
        print("Please make a selection.")
        group_select = input("1 - Coffee\n2 - Tea \n3 - Food \nd - Delete sale \nc - Clear sale \nq - Close sale \nb - Go Back \nInput: ")
        if group_select == '1':
            # If main selection is Coffee, make a subselection
            while True:
                print("Please select a Product:")
                product = int(input("1 - Black Coffee\n2 - Latte \n3 - Espresso Shot \nInput: "))
                if product == 1:
                    # If Product Type = Coffee
                    while True:
                        print("Please select a Product Type:")
                        i = 1
                        for key, value in coffee.items():
                            if "Coffee" in key:
                                print(f'{i} | {key} - {value}')
                                i += 1
                        item_select = int(input("Input: ")) - 1
                        # Validate the selected index
                        if 0 <= item_select < len(coffee):
                            selected_product = list(coffee.keys())[item_select]
                            selected_price = list(coffee.values())[item_select]
                            # Get user input for quantity
                            quantity = int(input("Quantity: "))
                            # Add the item to the sales
                            sale.addItem(item=selected_product, price=selected_price, quantity=quantity)
                            break
                        else:
                            # Checks for invalid input, if input is invalid, prompts user to try again
                            print("Invalid selection. Please try again.")
                        break
                    break
                elif product == 2:
                    # Product Type = Latte
                    while True:
                        print("Please select a Product Type:")
                        i = 1
                        for key, value in coffee.items():
                            if "Latte" in key:
                                print(f'{i} | {key} - {value}')
                                i += 1
                        item_select = int(input("Input: ")) + 2
                        # Validate the selected index and make sure contains relevant value
                        if 0 <= item_select < len(coffee):
                            selected_product = list(coffee.keys())[item_select]
                            selected_price = list(coffee.values())[item_select]
                            # Get user input for quantity
                            quantity = int(input("Quantity: "))
                            # Add the item to the sales
                            sale.addItem(item=selected_product, price=selected_price, quantity=quantity)
                            break
                        else:
                            print("Invalid selection. Please try again.")
                        break
                    break
                elif product == 3:
                    # Product Type = Espresso Shot
                    while True:
                        print("Please select a Product Type:")
                        i = 1
                        for key, value in coffee.items():
                            if "Shot" in key:
                                print(f'{i} | {key} - {value}')
                                i += 1
                        item_select = int(input("Input: ")) + 5
                        # Validate the selected index
                        if 0 <= item_select < len(coffee):
                            selected_product = list(coffee.keys())[item_select]
                            selected_price = list(coffee.values())[item_select]
                            # Get user input for quantity
                            quantity = int(input("Quantity: "))
                            # Add the item to the sales
                            sale.addItem(item=selected_product, price=selected_price, quantity=quantity)
                            break
                        else:
                            print("Invalid selection. Please try again.")
                        break
                    break
        elif group_select == '2':
            print("You have selected 2 - Tea")
            # IF TEA
            while True:
                print("Please select a Product:")
                product = int(input("1 - Black Tea\n2 - Green Tea \nInput: "))
                if product == 1:
                    # Product Type = Black Tea
                    while True:
                        print("Please select a Product Type:")
                        i = 1
                        for key, value in tea.items():
                            if "Black Tea" in key:
                                print(f'{i} | {key} - {value}')
                                i += 1
                        item_select = int(input("Input: ")) - 1
                        # Validate the selected index
                        if 0 <= item_select < len(tea):
                            selected_product = list(tea.keys())[item_select]
                            selected_price = list(tea.values())[item_select]
                            # Get user input for quantity
                            quantity = int(input("Quantity: "))
                            # Add the item to the sales
                            sale.addItem(item=selected_product, price=selected_price, quantity=quantity)
                            break
                        else:
                            print("Invalid selection. Please try again.")
                        break
                    break
                if product == 2:
                    # Product Type = Green Tea
                    while True:
                        print("Please select a Product Type:")
                        i = 1
                        for key, value in tea.items():
                            if "Green Tea" in key:
                                print(f'{i} | {key} - {value}')
                                i += 1
                        item_select = int(input("Input: ")) +2
                        # Validate the selected index
                        if 0 <= item_select < len(tea):
                            selected_product = list(tea.keys())[item_select]
                            selected_price = list(tea.values())[item_select]
                            # Get user input for quantity
                            quantity = int(input("Quantity: "))
                            # Add the item to the sales
                            sale.addItem(item=selected_product, price=selected_price, quantity=quantity)
                            break
                        else:
                            print("Invalid selection. Please try again.")
                        break
                    break
        elif group_select == '3':
            print("You have selected 3 - Food")
            # If selected Food
            # This section is unique since we don't have subcategories for food (black vs green tea).
            while True:
                print("Please select a Product Type:")
                i = 1
                for key, value in food.items():
                    print(f'{i} | {key} - {value}')
                    i += 1
                item_select = int(input("Input: ")) - 1
                # Validate the selected index
                if 0 <= item_select < len(food):
                    selected_product = list(food.keys())[item_select]
                    selected_price = list(food.values())[item_select]
                    # Get user input for quantity
                    quantity = int(input("Quantity: "))
                    # Add the item to the sales
                    sale.addItem(item=selected_product, price=selected_price, quantity=quantity)
                    break
                else:
                    print("Invalid selection. Please try again.")
                break
                
        elif group_select == 'c':
            sale.clearSale()
            break
            
        elif group_select == 'q':
            sale.closeSale(customer_df, sales_receipts_df, connSales)
            break
        
        elif group_select == 'b':
            selector()
            break

        elif group_select == 'd':
            if sale.orderSize >0:
                while True:
                    print("Please select an item to delete:")
                    i = 1
                    for key, value in sale.items.items():
                        print(f'{i} | {key} - {value}')
                        i += 1
                    item_select = int(input("Input: ")) - 1
                    # Validate the selected index
                    if 0 <= item_select < len(sale.items):
                        selected_product = list(sale.items.keys())[item_select]
                        selected_price = sale.items[selected_product]['price']
                        # Get user input for quantity
                        quantity = int(input("Quantity: "))
                        # Remove the item from the sales
                        sale.delItem(item=selected_product, price=selected_price, quantity=quantity)
                        break
                    else:
                        print("Invalid selection. Please try again.")
                    break
            else:
                print("You may not delete items from a sale that contains zero items.")
                startSale()
                break


        else:
            print("Invalid selection. Please input '1', '2', '3' or 'q' to close sale.")
            startSale()
            break

### 2.4 Sales Class

In [10]:
# #Cell #10       
class sales:
    def __init__(self, customer_df, product_df, sales_receipts_df, connSales):
        self.items = {}
        self.orderSize = 0
        self.orderPrice = 0

    def addItem(self, item, price, quantity=1):
        if item in self.items:
            # If item already in the self.items, update quantity and price to new values
            self.items[item]['quantity'] += quantity
            self.items[item]['price'] += price * quantity
        else:
            # If the item not in self.items, add it with the default quantity and price
            self.items[item] = {'quantity': quantity, 'price': price * quantity}

        self.orderSize += quantity
        self.orderPrice += price * quantity
        print(f'Added {quantity} {item} to sale.')
        print(self.getTotal())

    def delItem(self, item, price, quantity=1):
        if item in self.items:
        # If item already in self.items, update quantity and price to new values
            if self.items[item]['quantity'] >= quantity:
                original_price = price / self.items[item]['quantity']
                self.items[item]['quantity'] -= quantity
                self.orderSize -= quantity

                # Calculate the total price for the remaining quantity using the original price
                self.items[item]['price'] = original_price * self.items[item]['quantity']

                # Update the total order price
                self.orderPrice -= original_price * quantity

                if self.items[item]['quantity'] <= 0:
                    # If quantity becomes zero or negative, remove the item from the sale
                    del self.items[item]
            else:
                print(f"Error: Trying to remove more quantity than available for {item}")
            i = 1
            for key, value in self.items.items():
                print(f'{i} | {key} - {value}')
                i += 1

        else:
            print("Item not in sale")

    def clearSale(self):
        self.orderSize = 0
        self.orderPrice = 0
        self.items.clear()
        print(f'Sale cleared successfully')
        print(f'Current total: {self.orderPrice}')

    def getTotal(self, discount=1):
        print("=" * 20)
        for key, value in self.items.items():
            quantity = value['quantity']
            total_price = value['price']
            print(f'{quantity} {key}(s) - ${total_price:.2f}')
        print("=" * 20)
        print(f'Total Price: ${self.orderPrice * discount:.2f}')
        return (f'Total Items: {self.orderSize}\n')
    
    # Indicates that the sale is complete and transfers information into a sales receipt 
    def closeSale(self, customer_df, sales_receipts_df, connSales):
        phoneNum = input("Enter customer loyalty phone number (xxx-xxx-xxxx): ")
        file_path_sr = 'sales_reciepts.csv'
        last_transaction_id = sales_receipts_df['transaction_id'].iloc[-1]
        print(last_transaction_id)
        new_transaction_id = last_transaction_id + 1
        print(new_transaction_id)
        customerNum = pd.read_sql(f"""SELECT * 
                                      FROM Customer 
                                      WHERE loyalty_card_number = '{phoneNum}'""", connCustomer)
        # Gets the customer_id using the customer number that the user provided 
        cust_id = customerNum.iloc[0]['customer_id']
        
        # When customer loyalty points are at 9, a loyalty discount is applied and points are set to 0
        if customerNum.iloc[0]['points'] == 9:
            print("You've reached 10 visits! Enjoy a 5% discount!")
            print("=" * 20)
            print("FINAL RECEIPT:")
            # Discount can be configured here
            print(self.getTotal(discount=0.95))
            print (f"Sale #{new_transaction_id} closed succesfully.")
            customer_df.loc[customer_df['customer_id'] == cust_id, 'points'] = 0
            customer_df.to_csv('customer.csv', index = False)
            for key,value in self.items.items():
                product_df_row = product_df.loc[product_df['product'] == key]
                product_id = product_df_row.iloc[0]['product_id']

                quantity = value['quantity']
                unit_price = product_df_row.iloc[0]['current_retail_price'] 

                sale_to_csv = {
                'transaction_id': new_transaction_id,
                'transaction_date': str(datetime.now().date()),
                'customer_id': cust_id,
                'product_id': product_id,
                'quantity': quantity,
                'unit_price': unit_price,
                }

                sales_receipts_df = sales_receipts_df._append(sale_to_csv, ignore_index=True)
                sales_receipts_df.to_csv(file_path_sr, index = False)

                insert_statement = f"""
                                INSERT INTO SalesReceipts 
                                (transaction_id, transaction_date, customer_id, product_id, quantity, unit_price)
                                VALUES ('{new_transaction_id}', '{str(datetime.now().date())}', '{cust_id}', '{product_id}', '{quantity}', '{unit_price}')
                                """
                connSales = connSales.execute(insert_statement)
            selector()
            
        # If customer loyalty points aren't at 9 then the total is incremented by 1 
        elif customerNum.iloc[0]['points'] < 9:
            customer_df.loc[customer_df['customer_id'] == cust_id, 'points'] += 1
            customer_df.to_csv('customer.csv', index = False)
            
        # Each every product and value in the items list, creates a sales receipt for the product 
        for key,value in self.items.items():
            product_df_row = product_df.loc[product_df['product'] == key]
            product_id = product_df_row.iloc[0]['product_id']
            
            quantity = value['quantity']
            unit_price = product_df_row.iloc[0]['current_retail_price'] 
             
            sale_to_csv = {
            'transaction_id': new_transaction_id,
            'transaction_date': str(datetime.now().date()),
            'customer_id': cust_id,
            'product_id': product_id,
            'quantity': quantity,
            'unit_price': unit_price,
            }
            
            sales_receipts_df = sales_receipts_df._append(sale_to_csv, ignore_index=True)
            sales_receipts_df.to_csv(file_path_sr, index = False)
            
            insert_statement = f"""
                            INSERT INTO SalesReceipts 
                            (transaction_id, transaction_date, customer_id, product_id, quantity, unit_price)
                            VALUES ('{new_transaction_id}', '{str(datetime.now().date())}', '{cust_id}', '{product_id}', '{quantity}', '{unit_price}')
                            """
            connSales = connSales.execute(insert_statement)
        # Prints sales receipt
        print("=" * 20)
        print("FINAL RECEIPT:")
        print(self.getTotal())    
        print (f"Sale #{new_transaction_id} closed succesfully.")
        selector()

### 2.5. Loyalty Options

In [11]:
#Cell 11
# The loyalty options function can be called from the selector function
# Multiple options related to loyalty status and an option to return to the selector function 
def loyaltyOptions(connCustomer):
    while True:
        print("Type '1' to Lookup by Phone Number")
        print("Type '2' to Add New Loyalty Member")
        print("Type '3' to Go Back")
        action = int(input("Input an option (1, 2, or 3): "))

        
        # Action 1 looks up loyalty based on customer phone number 
        if action == 1:
            print("You have selected 1 - Lookup by Phone Number")
            phoneNum = input("Enter customer phone number (xxx-xxx-xxxx): ")
            customerNum = pd.read_sql(f"""SELECT * 
                                          FROM Customer 
                                          WHERE loyalty_card_number = '{phoneNum}'""", connCustomer)
            display(customerNum)
            selector()
            break
            
            
        # Action 2 adds a new loyalty member, inputing information needed     
        elif action == 2:
            print("You have selected 2 - Add New Loyalty Member")

            file_path = 'customer.csv'
            df = pd.read_csv(file_path)
            last_cust_id = df['customer_id'].iloc[-1]
            new_id = last_cust_id + 1
            lastName = input("Customer Last Name: ")
            custEmail = input("Customer Email: ")
            custPhoneNumber = input("Enter customer phone number (xxx-xxx-xxxx): ")
            custBirthdate = input("Enter customer DOB (xx/xx/xxxx): ")
            custGender = input("Enter customer gender (M or F): ")
            
            new_customer = {
            'customer_id' : new_id,
            'customer_firstname' : lastName,
            'customer_email' : custEmail,
            'customer_since' : datetime.now().date(),
            'loyalty_card_number' : custPhoneNumber,
            'birthdate' : custBirthdate,
            'gender' : custGender,
            'points' : 0
            }
        
            df = df._append(new_customer, ignore_index=True)
            df.to_csv(file_path, index = False)
            
            insert_statement = f"""
                            INSERT INTO Customer 
                            (customer_id, customer_firstname, customer_email, customer_since, loyalty_card_number, birthdate, gender, points)
                            VALUES ('{new_id}', '{lastName}', '{custEmail}', '{datetime.now().date()}', '{custPhoneNumber}', '{custBirthdate}', '{custGender}', 0)
                            """
            connCustomer = connCustomer.execute(insert_statement)
            print("Customer added successfully!")
            selector()
            break
        
        
        # Action 3 takes the user back to the selector function     
        elif action == 3:
            print("You have selected 3 - Go Back")
            selector()
            break
           
        
        # In the case of improper inputs, prompts the user to select one of the avaiable options    
        else:
            print("Please select option 1, 2, or 3.")


### 2.6 Management Options and Insight

In [12]:
#Cell 12
# The management options function can be called in the selector function
# Provides multiple options for managerial insights using sql queries, and provides the option to return to selector
def managementOptions():
     while True:
        print("Type '1' to View Frequently Visiting Customers")
        print("Type '2' to View Most Popular Items")
        print("Type '3' to View Purchase Behaviour")
        print("Type '4' to Go Back")
        action = int(input("Input an option (1, 2, 3 or 4): "))
        
        # Option 1 provides the top 5 most frequently visiting customers and how many times they have visited
        if action == 1:
            print("You have selected 1 - View Frequently Visiting Customers")
            print("The most frequent visiting customers by cust_id are: ")
            top_customer = pd.read_sql("""SELECT customer_id as Customer, COUNT(*)
                AS cust_count
                FROM SalesReceipts
                WHERE customer_id > 0
                GROUP BY customer_id
                ORDER BY cust_count DESC""", connSales).head(7) 
            custplt = top_customer.plot.bar(title = "Frequent Customers", color = ['red','gold','tan'], x='Customer', y='cust_count', rot=0)
            custplt.set_xlabel ("Customer ID")
            custplt.set_ylabel ("Purchase History")
            custplt.set_ylim(10,40)
            display(top_customer)
            plt.show()
            selector()
            break
        
        # Option 2 provides the top 5 most popular items and their counts
        elif action == 2:
            print("You have selected 2 - View Most Popular Items")
            print("The most popular items by Total Sales are: ")
            product_count = pd.read_sql("""SELECT product_id, COUNT(*)*quantity
                AS product_count
                FROM SalesReceipts
                GROUP BY product_id
                ORDER BY product_count DESC""", connSales).head(7)
            prdplt = product_count.plot.bar(title = "Popular Goods", color = ['red','gold','tan'] , x='product_id', y='product_count', rot=0)
            prdplt.set_xlabel ("Product ID")
            prdplt.set_ylabel ("Total Items")
            prdplt.set_ylim(1500,2500)
            display(product_count)
            plt.show()
            selector()
            break
            
        
        # Option 3 provides the monthly sales revenue for comparison purposes 
        elif action == 3:
            print("You have selected 3 - View Purchase Behaviour")
            print("This month's daily total sales revenue is: ")
            monthly_sales =  pd.read_sql("""SELECT DISTINCT transaction_date as Transaction_Date, sum(quantity * unit_price) as Total
                FROM SalesReceipts
                GROUP BY transaction_date""", connSales).head(30)
            saleschart = monthly_sales.plot.line(title = "Daily Monthly Sales", color = ['red','gold','tan'] , x='Transaction_Date', y='Total', rot=0)
            saleschart.set_xlabel ("Date")
            saleschart.set_ylabel ("Sales ($)")
            display(monthly_sales)
            plt.show()
            selector()
            break
            
            
        
        # Option 4 sends the user back to the selector function 
        elif action == 4:
            print("You have selected 4 - Go Back")
            selector()
            break
        
        # In the case of improper inputs, prompts the user to select one of the avaiable options
        else:
            print("Please select option 1, 2, 3, or 4.")

## 3. Table of Components

In [13]:
comp = pd.read_csv('component.csv')
print(comp)

        Functionality Location (Cell#)
0      Read/Write CSV               #1
1    User Interaction               #4
2  SQL table creation           #2, #3
3     Plot generation              #17


## 4. Implementation Result
The functionality of this project is to simulate a small coffee shop point-of-sale (POS) system that creates a new sale, has a loyalty member option, and has managerial insights to track trends throughout the months. Our sales creation consists of our menu on the user interface (UI) where the employee will choose what the customer tells them. At the end of the sale, they can close the sale and it will show the total price to communicate to the customer. From there, the data from the sale gets imported into our sales receipts csv file which is connected to our database. A customer can also use their loyalty account to gain points and receive a certain percentage discount after 10 points (currently set to 5%). The system can look up the customer in the system to see if they have a loyalty account and how many points they are at. If they would like to sign up for an account, the employee can grab their information and put it into the UI which will then import that data into our customer csv file which is connected to the database. Lastly, we have managerial insights that are generated using SQL to view frequently visiting customers, most popular items, and purchase behavior for the month. The overall impact of this POS system is to give the employees a more friendly UI to work with and a more structured system to view analytics based on products and customers.


## 5. Discussion
Overall the project was a valuable learning experience for everyone in the group. We all liked meeting up to figure out how to do the project and which roles each of us would take. In the group, we were able to find our roles and responsibilities pretty early on because two people in the group were well experienced in coding, while the other two were still able to code, but aided as help and provided insight for each part of the project. Without all of us working together it would have been much harder to finish the project and get it to run. We achieved all of our goals because they were based on the rubric and fulfilled all requirements. For possible improvements, if we had more time, we would have liked to condense some of the code to make it more efficient. We were also interested in adding additional features, such as converting customer_ids and product_ids into their name values as part of the managerial insight graphics, but time restrictions did not allow.

In [15]:
selector()

Please make a selection:
1 - Create New Sale 
2 - Loyalty Program 
3 - Management Options
4 - Shutdown
Input: 2
You have selected 2 - Loyalty Program
Type '1' to Lookup by Phone Number
Type '2' to Add New Loyalty Member
Type '3' to Go Back
Input an option (1, 2, or 3): 1
You have selected 1 - Lookup by Phone Number
Enter customer phone number (xxx-xxx-xxxx): 641-512-4404


,customer_id,customer_firstname,customer_email,customer_since,loyalty_card_number,birthdate,gender,points


Please make a selection:
1 - Create New Sale 
2 - Loyalty Program 
3 - Management Options
4 - Shutdown
Input: 2
You have selected 2 - Loyalty Program
Type '1' to Lookup by Phone Number
Type '2' to Add New Loyalty Member
Type '3' to Go Back
Input an option (1, 2, or 3): 2
You have selected 2 - Add New Loyalty Member
Customer Last Name: Kuhlemeier
Customer Email: 20coopkuhl@gmail.com
Enter customer phone number (xxx-xxx-xxxx): 641-512-4404
Enter customer DOB (xx/xx/xxxx): 04/29/2002
Enter customer gender (M or F): M
Customer added successfully!
Please make a selection:
1 - Create New Sale 
2 - Loyalty Program 
3 - Management Options
4 - Shutdown
Input: 2
You have selected 2 - Loyalty Program
Type '1' to Lookup by Phone Number
Type '2' to Add New Loyalty Member
Type '3' to Go Back
Input an option (1, 2, or 3): 1
You have selected 1 - Lookup by Phone Number
Enter customer phone number (xxx-xxx-xxxx): 641-512-4404


,customer_id,customer_firstname,customer_email,customer_since,loyalty_card_number,birthdate,gender,points
0,8502.0,Kuhlemeier,20coopkuhl@gmail.com,2023-12-05,641-512-4404,04/29/2002,M,0.0


Please make a selection:
1 - Create New Sale 
2 - Loyalty Program 
3 - Management Options
4 - Shutdown
Input: 1
You have selected 1 - Create New Sale
Please make a selection.
1 - Coffee
2 - Tea 
3 - Food 
d - Delete sale 
c - Clear sale 
q - Close sale 
b - Go Back 
Input: 1
Please select a Product:
1 - Black Coffee
2 - Latte 
3 - Espresso Shot 
Input: 1
Please select a Product Type:
1 | Small Black Coffee - 2.0
2 | Medium Black Coffee - 2.5
3 | Large Black Coffee - 3.0
Input: 1
Quantity: 1
Added 1 Small Black Coffee to sale.
1 Small Black Coffee(s) - $2.00
Total Price: $2.00
Total Items: 1

Please make a selection.
1 - Coffee
2 - Tea 
3 - Food 
d - Delete sale 
c - Clear sale 
q - Close sale 
b - Go Back 
Input: q
Enter customer loyalty phone number (xxx-xxx-xxxx): 641-512-4404
16830
16831
FINAL RECEIPT:
1 Small Black Coffee(s) - $2.00
Total Price: $2.00
Total Items: 1

Sale #16831 closed succesfully.
Please make a selection:
1 - Create New Sale 
2 - Loyalty Program 
3 - Management Op

In [ ]:
table = pd.read_sql('SELECT * FROM SalesReceipts ORDER BY transaction_id DESC LIMIT 10', connSales)
table